# Instalando dependências

In [17]:
%pip install pandas
%pip install joblib
%pip install tiktoken
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached matplotlib-3.9.2-cp310-cp310-win_amd64.whl (7.8 MB)
  Using cached kiwisolver-1.4.7-cp310-cp310-win_amd64.whl (55 kB)
     ---------------------------------------- 0.0/218.6 kB ? eta -:--:--
     ------ ------------------------------ 41.0/218.6 kB 960.0 kB/s eta 0:00:01
     ---------------------------- --------- 163.8/218.6 kB 1.6 MB/s eta 0:00:01
     -------------------------------------- 218.6/218.6 kB 1.9 MB/s eta 0:00:00
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
     ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
     ----- ---------------------------------- 0.3/2.2 MB 5.9 MB/s eta 0:00:01
     -------------- ------------------------- 0.8/2.2 MB 8.3 MB/s eta 0:00:01
     ------------------------ --------------- 1.4/2.2 MB 9.7 MB/s eta 0:00:01
     ---------------------------------------  2.2/2.2 MB 11.4 MB/s eta 0:00:01
     ---------------------------------------- 2.2/2.2 MB 10.8 MB/s eta 0:00:00
  Using cached pillow-11.0.0-cp310-c


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Carregando a base de dados dos simpsons

In [9]:
import pandas as pd

df_script = pd.read_csv('./data/thesimpsons/simpsons_script_lines.csv', low_memory=False)
df_episodes = pd.read_csv('./data/thesimpsons/simpsons_episodes.csv', low_memory=False)
df_characters = pd.read_csv('./data/thesimpsons/simpsons_characters.csv', low_memory=False)
df_locations = pd.read_csv('./data/thesimpsons/simpsons_locations.csv', low_memory=False)

df_script.set_index('id', inplace=True)
df_characters['id'] = df_characters['id'].astype(str)

df_characters = df_characters.add_prefix('character_')
df_locations = df_locations.add_prefix('location_')
df_episodes = df_episodes.add_prefix('episode_')

data = (
    df_script.merge(df_episodes, left_on='episode_id', right_on='episode_id')
             .merge(df_characters, left_on='character_id', right_on='character_id', how='left')
             .merge(df_locations, left_on='location_id', right_on='location_id', how='left')
)


assert data.shape[0] == df_script.shape[0]

In [ ]:
import joblib
import os
import pandas as pd
import google.generativeai as genai
from dotenv import load_dotenv

load_dotenv('../.env')


# Carga dos resumos
summaries = joblib.load('../data/results/simpsons_episode_summary.joblib')
print(summaries.keys())

data = pd.read_parquet('../data/results/database_thesimpsons.parquet')


data['line'] = ("Espisode " + data['episode_id'].astype(str) + ' | ' + 
                data['location_normalized_name'].fillna('') + ', ' + 
                data['character_normalized_name'].fillna('') + ' said: ' + 
                data['normalized_text'].fillna('')
)

print(data.columns)

# Estimativa de Tokens

In [14]:
import tiktoken

def get_tokens(text: str):
    encoder = tiktoken.get_encoding("cl100k_base")
    tokens = encoder.encode(text)
    return tokens


In [44]:
X = data.dropna(subset='normalized_text').copy()
X['n_tokens'] = X.normalized_text.fillna('').apply(lambda x: len(get_tokens(x)))
X.shape

(132087, 31)

# Calculando a média de tokens por episódio

In [47]:
token_per_episode = X.groupby('episode_id').n_tokens.sum()
general_token_mean = token_per_episode.mean()
most_tokens_episode = token_per_episode.idxmax()

print("O episódio com mais tokens é o", most_tokens_episode, "com", token_per_episode[most_tokens_episode], "tokens.")
print("A média de tokens por episódio é de {:.2f}".format(general_token_mean))

O episódio com mais tokens é o 49 com 3454 tokens.
A média de tokens por episódio é de 2606.98


# Calculando a média de tokens por temporada

In [ ]:
token_per_episode = X.groupby('episode_id').n_tokens.sum()
general_token_mean = token_per_episode.mean()
most_tokens_episode = token_per_episode.idxmax()

print("O episódio com mais tokens é o", most_tokens_episode, "com", token_per_episode[most_tokens_episode], "tokens.")
print("A média de tokens por episódio é de {:.2f}".format(general_token_mean))

O episódio com mais tokens é o 49 com 3454 tokens.
A média de tokens por episódio é de 2606.98
